In [ ]:
# Stress Testing — Market Risk Case Study

This notebook complements the VaR/CVaR analysis by evaluating the portfolio under historical stress events and scenario shocks.



In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Portfolio weights (same as in VaR notebook)
WEIGHTS = {
    "SPY": 0.20,
    "QQQ": 0.20,
    "IWM": 0.15,
    "TLT": 0.20,
    "GLD": 0.15,
    "HYG": 0.10
}

DATA_PATH = Path("../data/prices.csv")  # notebook is inside notebooks/


In [ ]:
prices = pd.read_csv(DATA_PATH)
prices["date"] = pd.to_datetime(prices["date"])
prices = prices.sort_values(["ticker", "date"]).reset_index(drop=True)

print("Rows, cols:", prices.shape)
print("Tickers:", sorted(prices["ticker"].unique()))
prices.head()


In [ ]:
prices["ret"] = prices.groupby("ticker")["adj_close"].pct_change()
returns = prices.dropna(subset=["ret"]).copy()

returns["weight"] = returns["ticker"].map(WEIGHTS)
missing = returns["weight"].isna().sum()
print("Missing weights:", missing)
returns.head()


In [ ]:
returns["w_ret"] = returns["ret"] * returns["weight"]

portfolio = (
    returns.groupby("date")["w_ret"].sum()
    .rename("portfolio_ret")
    .to_frame()
    .sort_index()
)

portfolio["loss"] = -portfolio["portfolio_ret"]
portfolio.head()


In [ ]:
## Stress Test A — Worst Historical Days (Single-Day Shocks)


In [ ]:
## Stress Test A — Worst Historical Days (Single-Day Shocks)
# Number of worst days to inspect
N = 10

worst_days = (
    portfolio
    .sort_values("loss", ascending=False)
    .head(N)
)

worst_days


In [ ]:
### A2. Inspect the single worst historical day
worst_day = worst_days.index[0]
worst_loss = worst_days.iloc[0]["loss"]

print(f"Worst historical day: {worst_day.date()}")
print(f"Portfolio loss: {worst_loss:.2%}")


In [ ]:
### A3. Asset-level contribution to loss on the worst day
contrib = (
    returns[returns["date"] == worst_day]
    .assign(weighted_ret=lambda x: x["ret"] * x["weight"])
    .loc[:, ["ticker", "weight", "ret", "weighted_ret"]]
    .sort_values("weighted_ret")  # most negative first
)

contrib


In [ ]:
### A4. Sanity check — portfolio return consistency
print(
    "Sum of weighted returns:",
    f"{contrib['weighted_ret'].sum():.4%}"
)

print(
    "Portfolio return:",
    f"{portfolio.loc[worst_day, 'portfolio_ret']:.4%}"
)


In [ ]:
## Stress Test B — COVID-19 Historical Scenario


In [ ]:
### B1. Define COVID stress window
covid_start = pd.to_datetime("2020-02-20")
covid_end = pd.to_datetime("2020-03-31")

covid_portfolio = portfolio.loc[covid_start:covid_end].copy()

covid_portfolio.head(), covid_portfolio.tail()


In [ ]:
### B2. Compute cumulative portfolio return during COVID window
covid_cum_return = (1 + covid_portfolio["portfolio_ret"]).prod() - 1

print("Cumulative portfolio return (COVID window):",
      f"{covid_cum_return:.2%}")


In [ ]:
### B3. Maximum drawdown during COVID window
cum_path = (1 + covid_portfolio["portfolio_ret"]).cumprod()
rolling_max = cum_path.cummax()
drawdown = cum_path / rolling_max - 1

max_drawdown = drawdown.min()

print("Maximum drawdown during COVID window:",
      f"{max_drawdown:.2%}")


In [ ]:
### B4. Worst single-day loss during COVID window
worst_covid_day = covid_portfolio["loss"].idxmax()
worst_covid_loss = covid_portfolio.loc[worst_covid_day, "loss"]

print("Worst COVID day:", worst_covid_day.date())
print("Worst daily loss:", f"{worst_covid_loss:.2%}")


In [ ]:
### B5. Drawdown visualization during COVID window
import matplotlib.pyplot as plt

# Recompute cumulative path (already used before, but explicit for clarity)
cum_path = (1 + covid_portfolio["portfolio_ret"]).cumprod()
rolling_max = cum_path.cummax()
drawdown = cum_path / rolling_max - 1

fig, ax = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# --- Top panel: cumulative portfolio value ---
ax[0].plot(cum_path, label="Cumulative portfolio value")
ax[0].plot(rolling_max, linestyle="--", label="Rolling peak")
ax[0].set_title("Portfolio cumulative performance (COVID window)")
ax[0].legend()
ax[0].grid(True)

# --- Bottom panel: drawdown ---
ax[1].plot(drawdown, label="Drawdown")
ax[1].axhline(drawdown.min(), linestyle="--", label=f"Max drawdown: {drawdown.min():.2%}")
ax[1].set_title("Drawdown")
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
### B1. Tariff Announcement Stress Window (April 2025)
### B1. Stress Test B – Trump Tariff Announcement (Multi-Day Shock, April 2025)

tariff_start = "2025-04-02"
tariff_end   = "2025-04-10"

tariff_portfolio = portfolio.loc[tariff_start:tariff_end]
tariff_portfolio


In [ ]:
### B2. Cumulative portfolio return during tariff shock

tariff_cum_return = (1 + tariff_portfolio["portfolio_ret"]).prod() - 1

print(
    "Cumulative portfolio return (tariff window):",
    f"{tariff_cum_return:.2%}"
)


In [ ]:
### B3. Cumulative portfolio return path during tariff shock

import matplotlib.pyplot as plt

# Cumulative return path
tariff_cum_path = (1 + tariff_portfolio["portfolio_ret"]).cumprod()

plt.figure(figsize=(10, 5))
plt.plot(tariff_cum_path, linewidth=2)
plt.axhline(1, linestyle="--", alpha=0.7)

plt.title("Cumulative Portfolio Return During Tariff Shock (Apr 2025)")
plt.ylabel("Portfolio Value (Start = 1)")
plt.xlabel("Date")

plt.show()


In [ ]:
### B4. Drawdown during tariff shock

rolling_max = tariff_cum_path.cummax()
tariff_drawdown = tariff_cum_path / rolling_max - 1

plt.figure(figsize=(10, 5))
plt.plot(tariff_drawdown, linewidth=2)
plt.axhline(0, linestyle="--", alpha=0.7)

plt.title("Portfolio Drawdown During Tariff Shock (Apr 2025)")
plt.ylabel("Drawdown")
plt.xlabel("Date")

plt.show()


In [ ]:
### B5. Maximum drawdown during tariff shock (numeric)

# Cumulative return path (already used in plots)
tariff_cum_path = (1 + tariff_portfolio["portfolio_ret"]).cumprod()

# Drawdown calculation
rolling_max = tariff_cum_path.cummax()
tariff_drawdown = tariff_cum_path / rolling_max - 1

# Maximum drawdown
max_tariff_drawdown = tariff_drawdown.min()

print(
    "Maximum drawdown during tariff shock:",
    f"{max_tariff_drawdown:.2%}"
)


In [ ]:
### Stress Test Summary — Historical vs COVID vs Tariff Shock
import pandas as pd

stress_summary = pd.DataFrame({
    "Scenario": [
        "Historical worst (single day)",
        "COVID crisis window",
        "Tariff shock (Apr 2025)"
    ],
    "Worst daily loss": [
        worst_days.iloc[0]["loss"],     # from Stress Test A
        worst_covid_loss,               # from COVID stress
        tariff_portfolio["loss"].max()  # from tariff window
    ],
    "Cumulative return": [
        worst_days.iloc[0]["portfolio_ret"],
        covid_cum_return,
        tariff_cum_return
    ],
    "Maximum drawdown": [
        worst_days.iloc[0]["loss"],     # single-day = drawdown
        max_drawdown,
        max_tariff_drawdown
    ]
})

# Format as percentages
stress_summary.set_index("Scenario").applymap(lambda x: f"{x:.2%}")
